In [1]:
import sys
from IPython.display import clear_output

sys.path.append("./matlab/myBTD2/myBTD2/for_redistribution_files_only")

import os

os.environ["LD_LIBRARY_PATH"] = "/home/renan/MATLAB/R2021b/runtime/glnxa64/"

import myBTD2
import numpy as np
import warnings
from tqdm import tqdm
import matlab
from scipy.linalg import hankel


warnings.filterwarnings("ignore", category=np.VisibleDeprecationWarning)


def hankelization(recording):
    _, n_lead = recording.shape

    T = []
    for i in range(0, n_lead):
        vec = recording[:, i]
        vec_filtered_hankel = hankel(vec)
        T.append(vec_filtered_hankel)

    T = np.array(T)
    T = matlab.double(T.tolist(), size=T.shape)

    return T


def get_btd(recording):
    btd = myBTD2.initialize()
    _, output = btd.myBTD2(
        5,
        matlab.double([17], size=(1, 1)),
        matlab.double([300], size=(1, 1)),
        matlab.double([1.0e-5], size=(1, 1)),
        matlab.double([1.0e-5], size=(1, 1)),
        matlab.double([False], size=(1, 1)),
        hankelization(recording),
        nargout=2,
    )
    btd.terminate()

    keys_to_extract = [
        "iter",
        "fval",
        "relfval",
        "relstep",
        "delta",
        "rho",
        "relerr",
    ]

    object_output = [output[key] for key in keys_to_extract]

    return object_output

In [2]:
import numpy as np

alias_db = "NSRDB"

if not os.path.exists(alias_db):
    os.makedirs(alias_db)

filename = f"{alias_db}/16273_0_200_result.npy"

samples = np.load(f"../extract_chunks/{filename}", allow_pickle=True)

recording_result = np.empty((0, 57))

for sample in tqdm(samples):
    rri_segment, recording = sample

    rri_histogram, _ = np.histogram(a=rri_segment, range=(0, 2500), bins=50)

    if rri_histogram.shape[0] == 0:
        continue

    btd = get_btd(recording)
    clear_output(wait=True)

    combine = np.hstack((rri_histogram, np.array(btd)))

    recording_result = np.vstack((recording_result, combine))

np.save(
    file=filename,
    arr=recording_result,
)

100%|██████████| 200/200 [1:20:29<00:00, 24.15s/it]
